In [5]:
from sagas.nlu.uni_intf import RootWordImpl, WordIntf, SentenceIntf

rel_mapping={'hed': 'root', 'sbv': 'nsubj',
             'vob': 'obj', 'iob': 'iobj', 'fob': 'obj', 'pob': 'obl',
            }
class LtpWordWrapper(WordIntf):
    def setup(self, token):
        deprel=token['dependency_relation']
        token['dependency_relation']=rel_mapping[deprel] if deprel in rel_mapping else deprel
        token['dependency_relation_raw']=deprel
        return token

class LtpSentWrapper(SentenceIntf):
    def setup(self, json_words):
        words = []
        for word in json_words:
            words.append(LtpWordWrapper(word))
        return words, []
    
from sagas.nlu.anal import build_anal_tree, Doc
from sagas.nlu.uni_jsonifier import JsonifySentImpl
from anytree.search import findall, findall_by_attr

f=build_anal_tree('附近有什么好吃的饭馆', 'zh', 'ltp', None, LtpSentWrapper)
f.draw()
# f.doc.predicts    
f.resolve_rel("obj").text, \
    f.resolve_rels("obj*")[0].text

root: 有 (有, verb)
|-- adv: 附近 (附近, noun)
+-- obj: 饭馆 (饭馆, noun)
    |-- att: 什么 (什么, pron)
    +-- att: 好吃 (好吃, adj)
        +-- rad: 的 (的, part)


('饭馆', '饭馆')

In [19]:
f.doc.predicts

[{'type': 'predicate',
  'lemma': '有',
  'index': 1,
  'phonetic': 'you',
  'word': '有',
  'rel': 'hed',
  'governor': 0,
  'pos': 'verb',
  'domains': [['a0', 0, '附近', '附近', ['附近'], ['c_noun', 'x_nd']],
   ['a1', 5, '饭馆', '饭馆', ['什么', '好吃', '的', '饭馆'], ['c_noun', 'x_n']]],
  'stems': []}]

In [12]:
import sagas
import pandas as pd
import sagas.util.pandas_helper as ph

r=f.doc.predicts[0]
df = sagas.to_df(r['domains'], ['rel', 'index', 'text', 'lemma', 'children', 'features'])
display(df)
ph.to_json(df)

,rel,index,text,lemma,children,features
0,a0,0,附近,附近,[附近],"[c_noun, x_nd]"
1,a1,5,饭馆,饭馆,"[什么, 好吃, 的, 饭馆]","[c_noun, x_n]"


[{'rel': 'a0',
  'index': 0,
  'text': '附近',
  'lemma': '附近',
  'children': ['附近'],
  'features': ['c_noun', 'x_nd']},
 {'rel': 'a1',
  'index': 5,
  'text': '饭馆',
  'lemma': '饭馆',
  'children': ['什么', '好吃', '的', '饭馆'],
  'features': ['c_noun', 'x_n']}]

In [22]:
from typing import Text, Any, Dict, List, Union, Optional, Tuple, Set
from dataclasses import dataclass
from dataclasses_json import dataclass_json

@dataclass_json
@dataclass
class SemanticRole:
    rel: str
    index: int
    text: str
    lemma: str
    children: List[Text]
    features: List[Text]

@dataclass_json
@dataclass
class Predict:
    type: str
    lemma: str
    index: int
    phonetic: str
    word: str
    rel: str
    governor: int
    pos: str
    domains: List[SemanticRole]

preds=[]
for p in f.doc.predicts:
    pred=Predict(type=p['type'], lemma=p['lemma'], index=p['index'],
                 phonetic=p['phonetic'], word=p['word'], 
                 rel=p['rel'], governor=p['governor'],
                 pos=p['pos'], domains=[]
                )
    df = sagas.to_df(p['domains'], ['rel', 'index', 'text', 'lemma', 'children', 'features'])
    
    for e in ph.to_json(df):
        pred.domains.append(SemanticRole(rel=e['rel'], index=e['index'], text=e['text'],
                     lemma=e['lemma'], children=e['children'], 
                     features=e['features']
                    ))
    preds.append(pred)
preds

[Predict(type='predicate', lemma='有', index=1, phonetic='you', word='有', rel='hed', governor=0, pos='verb', domains=[SemanticRole(rel='a0', index=0, text='附近', lemma='附近', children=['附近'], features=['c_noun', 'x_nd']), SemanticRole(rel='a1', index=5, text='饭馆', lemma='饭馆', children=['什么', '好吃', '的', '饭馆'], features=['c_noun', 'x_n'])])]

In [3]:
from sagas.nlu.uni_intf import RootWordImpl, WordIntf, SentenceIntf
import sagas
from sagas.nlu.utils import fix_sents

rel_mapping={'D': 'att', '時間': 'obl:time', 
             'ガ': 'nsubj', 'ヲ': 'obj',
             'デ': 'obl', 'ニ':'obl',
            }
class KnpWordWrapper(WordIntf):
    def setup(self, token):
        deprel=token['dependency_relation']
        token['dependency_relation']=rel_mapping[deprel] if deprel in rel_mapping else deprel
        token['dependency_relation_raw']=deprel
        return token

class KnpSentWrapper(SentenceIntf):
    def setup(self, json_words):
        words = []
        for word in json_words:
            words.append(KnpWordWrapper(word))
        return words, []
    
from sagas.nlu.anal import build_anal_tree, Doc
sents=["下鴨神社の参道は暗かった。", 
       '2008年12月に上海に行きたいです。',
       "望遠鏡で泳いでいる少女を見た。",
       'アテネ行きの 便を 予約したいの です が 。',
       '直行便 ですか ？',
       '次の ローマ行きは 何時 です か ？',
       '荷物は どれくらい 持って いけます か ？',
       '家へ 帰りたい です 。',
       'メニューを お願い します 。',
       'コーヒーを ミルク付きで お願い します 。',
      ]
for i,s in enumerate(sents):
    # s=fix_sents(s,'ja')    
    f=build_anal_tree(s, 'ja', 'knp', None, KnpSentWrapper)
    print(i,f.doc.sents)
    f.draw()

0 下鴨神社の参道は暗かった。
root: 暗かった。 (暗い, adj)
+-- nsubj: 参道は (参道, noun)
    +-- att: 神社の (神社, noun)
        +-- att: 下鴨 (下鴨, noun)
1 2008年12月に上海に行きたいです。
root: 行きたいです。 (行く, verb)
|-- obl:time: 12月に (12, num)
|   +-- att: 2008年 (2008, num)
+-- obl: 上海に (上海, noun)
2 望遠鏡で泳いでいる少女を見た。
root: 見た。 (見る, verb)
|-- obl: 鏡で (鏡, noun)
|   +-- att: 望遠 (望遠, noun)
+-- obj: 少女を (少女, noun)
    +-- nsubj: 泳いでいる (泳ぐ, verb)
3 アテネ行きの便を予約したいのですが。
root: 予約したいのですが。 (予約, noun)
+-- obj: 便を (便, noun)
    +-- att: 行きの (行き, x)
        +-- att: アテネ (アテネ, noun)
4 直行便ですか？
root: 便ですか？ (便, noun)
+-- att: 直行 (直行, noun)
5 次のローマ行きは何時ですか？
root: 何時ですか？ (何, num)
+-- nsubj: 行きは (行き, x)
    |-- att: 次の (次の, det)
    +-- att: ローマ (ローマ, noun)
6 荷物はどれくらい持っていけますか？
root: 持っていけますか？ (持つ, verb)
|-- obj: 荷物は (荷物, noun)
+-- nsubj: どれくらい (どれ, x)
7 家へ帰りたいです。
root: 帰りたいです。 (帰る, verb)
+-- obl: 家へ (家, noun)
8 メニューをお願いします。
root: お願いします。 (願う, x)
+-- obj: メニューを (メニュー, noun)
9 コーヒーをミルク付きでお願いします。
root: お願いします。 (願う, x)
|-- obj: コーヒーを (珈琲, noun)
+-- obl: 付きで

In [47]:
from sagas.nlu.anal import build_anal_tree, Doc
from sagas.nlu.uni_jsonifier import JsonifySentImpl
from anytree.search import findall, findall_by_attr

f=build_anal_tree('2008年12月に上海に行きたいです。', 'ja', 'knp', None, KnpSentWrapper)
f.draw()
r=f.doc.predicts[0]
df = sagas.to_df(r['domains'], ['rel', 'index', 'text', 'lemma', 'children', 'features'])
display(df)
f.resolve_rel("obl").text, \
    f.resolve_rels("obl*")[0].text

root: 行きたいです。 (行く, verb)
|-- obl:time: 12月に (12, num)
|   +-- att: 2008年 (2008, num)
+-- obl: 上海に (上海, noun)


,rel,index,text,lemma,children,features
0,ニ,2,上海に,上海,[上海],"[c_noun, x_c]"
1,時間,1,12月に,月,"[12, 月]","[c_num, x_n]"


('上海に', '12月に')

In [48]:
from sagas.nlu.anal import build_anal_tree, Doc
f=build_anal_tree("望遠鏡で泳いでいる少女を見た。", 'ja', 'knp', None, KnpSentWrapper)
f.draw()

root: 見た。 (見る, verb)
|-- obl: 鏡で (鏡, noun)
|   +-- att: 望遠 (望遠, noun)
+-- obj: 少女を (少女, noun)
    +-- nsubj: 泳いでいる (泳ぐ, verb)


In [3]:
from sagas.nlu.anal import build_anal_tree, Doc
sents=["下鴨神社の参道は暗かった。"]
for s in sents:
    f=build_anal_tree(s, 'ja', 'knp')
    f.draw()
    print(f.doc.predicts)
    list(map(display, f.doc.predict_domains_df))

root: 暗かった。 (暗い, adj)
+-- nsubj: 参道は (参道, noun)
    +-- att: 神社の (神社, noun)
        +-- att: 下鴨 (下鴨, noun)
[Predict(type='predicate', lemma='暗い', index=3, phonetic='くらい', word='暗かった。', rel='D', governor=0, pos='adj', domains=[SemanticRole(rel='ガ', index=2, text='参道は', lemma='参道', children=['参道'], features=['c_noun', 'x_n'])])]


,rel,index,text,lemma,children,features
0,ガ,2,参道は,参道,[参道],"[c_noun, x_n]"
